In [1]:
#API Key 313ee463bb714ba58a9e6b78f6782b21

!pip install schedule==1.1.0

import schedule
import time
import datetime
import requests
import json
import pandas as pd
import os


def fetch_bls_data():

    today = datetime.datetime.today()

    # Check if today is the 13th day of the month
    if today.day == 13:

      # Series IDs go here along with my custom timeframe thats dynamic
      series_ids = ['CES0000000001','LNS14000000','CUUR0000SA0','WPUFD4'] #Nonfarm Payroll, Unemployment, CPI-U, PPI

      # Calculate the start and end year; preperation for gather rolling 12 months of data
      start_year = today.year - 1
      start_month = today.month - 1
      if start_month == 1:
        start_month = 12
        start_year = today.year - 1

      end_year = today.year
      end_month = today.month

      # Data being requested
      data = json.dumps({"seriesid": series_ids, "startyear": str(start_year), "endyear": str(end_year)})
      headers = {'Content-type': 'application/json'}

      # Make the API request
      response = requests.post('https://api.bls.gov/publicAPI/v1/timeseries/data/', data=data, headers=headers)

      # Check if the request was successful
      if response.status_code == 200:
          # Parse the JSON response
          data = json.loads(response.text)

          # Extract data into a list of dictionaries
          all_data = []
          print(data)
          month_string = ""
          #compare item ['period'] with the intended start_month
          if start_month < 10:
            month_string = "M0" + str(start_month)
          else:
            month_string = "M" + str(start_month)
          for series in data['Results']['series']:
              series_id = series['seriesID']
              for item in series['data']:
                if item['year'] == str(start_year) and item['period'] < month_string:
                  continue
                else:
                  row = {
                      'seriesID': series_id,
                      'year': item['year'],
                      'period': item['period'],
                      'value': item['value']
                  }
                  all_data.append(row)

          # Create a Pandas DataFrame
          df = pd.DataFrame(all_data)
          print(df)

          # Load existing data from CSV, if it exists
          file_path = 'bls_data.csv'

          df.to_csv(file_path, index=False)

          print(f"Data fetched and stored to {file_path} successfully!")

      else:
          print(f"Error fetching data: {response.status_code}")
    else:
      print("Not the 8th day of the month. Skipping data fetch.")

fetch_bls_data()

schedule.every().day.at("02:00").do(fetch_bls_data)

# Keep the script running to execute the scheduled task
while True:
    schedule.run_pending()
    time.sleep(1)

Not the 8th day of the month. Skipping data fetch.


KeyboardInterrupt: 

In [2]:
!pip install streamlit
import streamlit as st
import pandas as pd
import plotly.express as px
import os

# Load data from CSV
@st.cache
def load_data():
    # Try to load data from the CSV, if it exists
    file_path = 'bls_data.csv'
    if os.path.exists(file_path):
        df = pd.read_csv(file_path)
        return df
    else:
        st.error("Data file 'bls_data.csv' not found!")
        return None

# Fetch and clean the data
def prepare_data(df):
    # Convert the 'year' and 'period' columns to strings for easy comparison
    df['date'] = df['year'].astype(str) + '-' + df['period'].apply(lambda x: f'{int(x):02}')
    df['date'] = pd.to_datetime(df['date'], format='%Y-%m')

    # Convert 'value' to numeric, forcing errors to NaN (useful for any non-numeric values)
    df['value'] = pd.to_numeric(df['value'], errors='coerce')

    return df

# Filter and plot the data
def plot_data(df):
    # Filter data for Unemployment, Nonfarm Payroll, CPI-U, PPI
    unemployment_df = df[df['seriesID'] == 'LNS14000000']
    payroll_df = df[df['seriesID'] == 'CES0000000001']
    cpi_df = df[df['seriesID'] == 'CUUR0000SA0']
    ppi_df = df[df['seriesID'] == 'WPUFD4']

    # Set up a multi-tab layout in Streamlit
    st.title('BLS Data Dashboard')

    # Line Chart for Unemployment and Nonfarm Payroll
    st.subheader('Unemployment and Nonfarm Payroll over Time')
    st.write('This line chart shows the Unemployment and Nonfarm Payroll data over the last 12 months.')

    # Prepare data for Plotly Line Chart
    unemployment_df_sorted = unemployment_df.sort_values(by='date')
    payroll_df_sorted = payroll_df.sort_values(by='date')

    # Combine the data for easier plotting
    line_data = pd.concat([unemployment_df_sorted[['date', 'value']],
                           payroll_df_sorted[['date', 'value']].rename(columns={'value': 'Nonfarm Payroll'})],
                          axis=1)
    line_data.columns = ['date', 'Unemployment', 'Nonfarm Payroll']

    # Create the Line Chart with Plotly
    fig_line = px.line(line_data, x='date', y=['Unemployment', 'Nonfarm Payroll'],
                       title="Unemployment and Nonfarm Payroll over Time",
                       labels={'date': 'Date', 'value': 'Value'},
                       template='plotly_dark')

    st.plotly_chart(fig_line)

    # Bar Chart for CPI-U and PPI
    st.subheader('CPI-U and PPI over Time')
    st.write('This bar chart shows the Consumer Price Index for All Urban Consumers (CPI-U) and Producer Price Index (PPI) over the last 12 months.')

    # Prepare data for Plotly Bar Chart
    cpi_df_sorted = cpi_df.sort_values(by='date')
    ppi_df_sorted = ppi_df.sort_values(by='date')

    # Combine the data for easier plotting
    cpi_ppi_df = pd.merge(cpi_df_sorted[['date', 'value']], ppi_df_sorted[['date', 'value']], on='date', suffixes=('_CPI', '_PPI'))
    cpi_ppi_df.set_index('date', inplace=True)

    # Create the Bar Chart with Plotly
    fig_bar = px.bar(cpi_ppi_df, x=cpi_ppi_df.index, y=['value_CPI', 'value_PPI'],
                     title="CPI-U and PPI over Time",
                     labels={'value_CPI': 'CPI-U', 'value_PPI': 'PPI'},
                     template='plotly_dark')

    st.plotly_chart(fig_bar)

# Main Streamlit app function
def main():
    # Load the data
    df = load_data()

    if df is not None:
        df = prepare_data(df)
        plot_data(df)

if __name__ == "__main__":
    main()


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 53.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 65.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.2 MB/s eta 0:00:00


2024-12-14 17:26:13.218 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-14 17:26:13.332 
  command:

    streamlit run /usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2024-12-14 17:26:13.335 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-14 17:26:13.339 
`st.cache` is deprecated and will be removed soon. Please use one of Streamlit's new
caching commands, `st.cache_data` or `st.cache_resource`. More information
[in our docs](https://docs.streamlit.io/develop/concepts/architecture/caching).

**Note**: The behavior of `st.cache` was updated in Streamlit 1.36 to the new caching
logic used by `st.cache_data` and `st.cache_resource`. This might lead to some problems
or unexpected behavior in certain edge cases.

2024-12-14 17:26:13.342 No runtime found, using MemoryCacheStorageManager
2024-12-14 17:26:13.350 No runtime found, using MemoryC